In [ ]:
'''CODE TO FIND CLOSURE OF FUNCTIONAL DEPENDENCIES'''
from xml.dom.minidom import Element
import re
import itertools 

n = int(input("Enter the number of elements:\n"))
li = []
for i in range(n):
    li.append(input("Enter the element:\n "))
m = int(input("Enter the number of functional dependencies:\n"))
dic = {}

for i in range(m):
    k = input("Enter the element:\n")
    j = input("Enter its dependency:\n")
   
    
    if dic.get(k):
        dic[k] += j
    else:
        dic[k] = j

li_set = [] #for unique attributes of the scehme we need to store in the set
for e in li:
    li_set.append(set(e))

def upd (set_ele, dic):
    old = set_ele
    for k in dic:
        if (set(k).issubset(set_ele)):
            set_ele = set_ele | set(dic[k])
    while (old != set_ele):
        old = set_ele
        for k in dic:
            if (set(k).issubset(set_ele)):
                set_ele = set_ele | set(dic[k])
    ans = ""
    for i in set_ele:
        ans += i
    return ans
    
ans = {}
for i in range(len(li_set)):
    ans[li[i]] = upd (li_set[i], dic)
print()
print("Closure:")
print(ans)




Enter the number of elements:
5
Enter the element:
 A
Enter the element:
 B
Enter the element:
 C
Enter the element:
 D
Enter the element:
 E
Enter the number of functional dependencies:
4
Enter the element:
A
Enter its dependency:
BC
Enter the element:
CD
Enter its dependency:
E
Enter the element:
B
Enter its dependency:
D
Enter the element:
E
Enter its dependency:
A

Closure:
{'A': 'DBCAE', 'B': 'DB', 'C': 'C', 'D': 'D', 'E': 'DBCAE'}


In [1]:
from itertools import chain, combinations

def powerset(iterable):   
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))  #R--> HOW MANY ATTRIBUTES ARE NEEDED IN THE COMBINATION

def compute_closure(X, F): #X-->ATTRIBUTE
    X_plus = X
    change = True
    while change:
        change = False
        for w,z in F:
            if type(w) == tuple:
                w_set = set(w)
            else:
                w_set = {w}
            if type(z) == tuple:
                z_set = set(z)
            else:
                z_set = {z}            
            if w_set.issubset(X_plus) \
                and not z_set.issubset(X_plus):
                X_plus = X_plus.union(z_set)
                change = True
    return X_plus

def compute_candidate_key(X, R, F): #X IS SUPERKEY OR CANDIDATE KEY CHECKING
    # X is a superkey
    T = set()
    remove = True
    while remove:
        for i in X: #LET'S REMOVE ONE ATTRIBUTE FROM SUPERKEY AND CHECK IF IT'S POTENTIAL CK 
            X_inter = compute_closure(X.difference({i}), F)#CHECK THE MINIMAL SUBSET FROM SK FOR CK
            if X_inter == R:
                X = X.difference({i})
            else:
                remove = False #SO THAT SP IS NOW CK AS ALL ATTRIBUTES ARE NEEDED
    T = T.union(X)#ADD THAT CK IN THE SET
    return T

def compute_all_candidate_keys(R, F):
    candidate_keys = []
    F_LHS = set()
    F_RHS = set()
    for w, z in F:  #w-->left side, z-->right side
        if type(w) == tuple:
            w_set = set(w)
        else:
            w_set = {w}
        if type(z) == tuple:
            z_set = set(z)
        else:
            z_set = {z}
        F_LHS = F_LHS.union(w_set)
        F_RHS = F_RHS.union(z_set)
    powerset_X = list([set(x) for x in powerset(F_LHS)])  #for making combination of all the left and middle keys
    for s in powerset_X:
        # whether set s is a superkey
        if compute_closure(s,F) == R:
            X_superkey = s
            # closure = R and not in candidate key ---> a potential candidate key
            if compute_closure(X_superkey, F) == R \
                and X_superkey not in candidate_keys:
                candidate_keys.append(
                   compute_candidate_key(X_superkey, R, F))
    candidate_keys = set([frozenset(x)   #FROZENSET-->FIXED SET[IMMUTABLE]
                          for x in candidate_keys 
                          if x != set()])
    return candidate_keys

if __name__ == '__main__':

    # F = {('A','B'),(('B', 'C'),'D'),('A','C')}
    # R = {'A', 'B', 'C', 'D'}
    
    R_2 = {'A', 'B', 'C', 'D', 'E','F'}
    F_2 = {('C','D'),(('A','B'),'C'),('C','E'),('E','F'),('F','A')}
    
    # R_1 = {'A', 'B', 'C', 'D','E','F','G','H'}
    # F_1 = { ('E','A'),(('C','H'),'G') ,('A','B'),('A','C'), ('B','C'),('B','F'),('B','H'),('F','E'), ('F','G') }  #d-->a, ab-->cd

#     print(compute_all_candidate_keys(R_2, F_2))
    print("The candidate keys are: ")
    print(compute_all_candidate_keys(R_2, F_2))
   # print(compute_all_candidate_keys(R_1, F_1))

    # print(compute_closure(set(('A','B')), F_1))
    # print(compute_closure(set('D'), F_1))





The candidate keys are: 
{frozenset({'C', 'B'}), frozenset({'A', 'B'}), frozenset({'F', 'B'}), frozenset({'E', 'B'})}


In [ ]:
import itertools
import random

def closure(R, FD, S):
    '''
    Check each functional dependency X -> Y on each iteration and add Y to the closure iff 
    X is a subset of the closure formed so far, repeat until there is no change in the set
    '''
    if not (is_subset(S, R)):
        return [] 
    result = list(S)
    changed = True
    while (changed):
        changed = False
        for fd in FD:
            LHS = fd[0]
            RHS = fd[1]
            if (is_subset(LHS, result)):
                for att in RHS:
                    if (att not in result):
                        result.append(att)
                        changed = True
    return sorted(result)


def all_closures(R, FD): 
    '''
    Calculate closure of each attribute susbset of the schema unless the subset is superkey 
    but not candidate key. Check in each iteration if the attribute closure equals to R, 
    that is, check for candidate key, and if it is not minimal then just continue 
    with the next iteration instead of appending closure to the result set
    '''
    result = []
    candidate_keys = []
    for att_set in subsets(R):
        att_closure = closure(R, FD, att_set)
        if (att_closure == R):
            is_super = False
            for key in candidate_keys:
                if is_subset(key, att_set):
                    is_super = True
            if is_super:
                continue
            candidate_keys.append(att_set)
        result.append([att_set, att_closure])
    return result

##  Return all minimal covers reachable from the functional dependencies of a given schema R and functional dependencies F.
def min_covers(R, FD):
    '''
    1) Simplfy right side of each FD, i.e. X -> YZ becomez X -> Y and X -> Z
    2) Simplify left side of each FD i.e. XY -> Z and if X already haz Z on its closure, 
    then X -> Z, or vice versa X -> Y, XY -> Z otherwise
    Put into a list of all different products of the possibilities
     which can be vary according to choice of X -> Z or X -> Y or XY -> Z
    3) Eliminate duplicate FDs first, then check for redundant FDs and eliminate them, 
    finally eliminate extraneous attributes on each FD if exists
    '''
    FD1 = find_sigma1(FD)
    FD2 = find_sigma2(R, FD, FD1)
    FD3 = []
    for fd in FD2:
        FD3.append(find_sigma3(R, fd))
    return FD3

## Helper functions
def is_subset(X, Y):
    '''
    Check if the list is subset of another list
    '''
    return set(X).issubset(set(Y)) 

def subsets(R):
    '''
    Calculate all possible subsets of given list
    Return them in a sorted way (first alphabetic and then length)
    '''
    x = len(R)
    masks = [1 << i for i in range(x)]
    result = []
    for i in range(1, 1 << x):
        r = []
        for mask, ss in zip(masks, R):
            if i & mask:
                r.append(ss)
        result.append(r)  
    result.sort() 
    result.sort(key=len) 
    return result

def find_sigma1(FD): #singelton RHS
    '''
    Divide RHS of each FD and add the products to the new FD set, 
    i.e. {X -> YZ, X -> Z} -> {X -> Y, X -> Z, X -> Z}
    '''
    FD1 = []
    for fd in FD:
        rhs_size = len(fd[1])
        if (rhs_size <= 1):
            FD1.append(fd)
        else:
            for att in fd[1]:
                FD1.append([fd[0], [att]])
    return FD1


def find_sigma2(R, FD, FD1):
    '''
    Simplify left side of each FD i.e. XY -> Z and if X already haz Z on its closure,
     then X -> Z, or vice versa X -> Y, XY -> Z otherwise
    Algorithm first decide all possible outcome of each FD and put them into a list,
     then take cross product of all lists in the formed list
    '''
    all_possible_fds = []
    FD2 = []
    for fd in FD1:
        possible_fds = []
        lhs_size = len(fd[0])
        if (lhs_size <= 1):
                possible_fds.append(fd)
        else:
            found = False
            for att in fd[0]:
                if fd[1][0] in closure(R, FD, [att]):
                    found = True
                    possible_fds.append([[att], [fd[1][0]]])
            if not found:
                possible_fds.append(fd)
        all_possible_fds.append(possible_fds)

    for tup in itertools.product(*all_possible_fds):
        FD2.append(list(tup))
    return FD2


def find_sigma3(R, FD2):
    '''
    1) Remove all duplicates remaining from sigma2
    2) Remove all redundant fds by checking iff 
    the rhs still implied in the absence of fd, if yes remove
    3) Remove extraneous attributes by checking iff 
    the closure is still contains the attribute within the absence of attribute
    '''
    FD3 = remove_duplicates(FD2)
    FD3 = remove_redundant_fd(R, FD3)
    FD3 = remove_extraneous_att(R, FD3)

    return FD3

def remove_duplicates(FD):
    '''
    Remove all duplicates from the fd set
    '''
    FD2 = list(FD)
    FD2.sort()
    FD2 = list(FD2 for FD2,_ in itertools.groupby(FD2))  #a->b,a->c, b->c, b->d == a->bc, b->cd
    return FD2

def remove_redundant_fd(R, FD): # a->b, b->c == a->c
    '''
    Remove all redundant fds by checking if the rhs still 
    implied in the absence of fd, if yes remove
    '''
    for fd in FD:
        FD2 = list(FD)
        FD2.remove(fd)
        if is_subset(fd[1], closure(R, FD2, fd[0])):
            FD.remove(fd)
    return FD

def remove_extraneous_att(R, FD): 
    '''
    Remove extraneous attributes by checking if the closure is 
    still contains the attribute within the absence of attribute
    '''
    for fd in FD:
        for att in fd[0]:
            if is_subset(att, closure(R, FD, [n for n in fd[0] if n != att])):  # xy->z, x->z == x->z
                fd[0].remove(att)   
    return FD

## Main functions
def main():

#     R = ['A', 'B', 'C', 'D', 'E']
#     FD = [[['B'], ['A']], [['A', 'C'], ['B', 'D', 'E']]]

  # R = ['A', 'B', 'C', 'D']
    # FD = [   [['A'], ['B']] ,  [['C'], ['B']] , [['D'], ['A', 'B', 'C']] ,  [  ['A','C'] ,['D'] ]    ]  #a->b, c->b, d->abc, ac->d
#     print(all_closures(R, FD)) 

    R = ['A', 'B', 'C', 'D', 'E']
    FD = [[['A'], ['B']], [['A','B'], ['C']],[['D'], ['A','C']],[['D'], ['E']]]
    
    
    print(min_covers(R,FD))
  






if __name__ == '__main__':
    main()

[[[['A'], ['B']], [['A'], ['C']], [['D'], ['A']], [['D'], ['E']]]]
